<a href="https://colab.research.google.com/github/Zolinad/Proc_Imag_-_Vis_Comp/blob/main/AULA10/treinar_modelo_cvkeras_003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Treinamento de Rede Neural Convolucional para Reconhecimento e Classificação de Faces**

In [ ]:
# O dataset deve ser zipado e ser arrastado para o Colab
#
# O dataset zipado deve ser descompactado na pasta local do Colab com o seguinte comando:
!unzip /content/dataset.zip -d /content/

In [11]:
# Importando bibliotecas necessárias
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Definindo o gerador de dados (DIVERSIFICANDO os dados, ao variar as imagens)
datagen = ImageDataGenerator(
    rescale=1./255,         # Normalização
    validation_split=0.2,   # Separar 20% para validação (sobram 80% para treino)
    height_shift_range=0.2, # Transladar verticalmente até 20% da altura
    width_shift_range=0.2,  # Transladar horizontalmente até 20% da largura
    rotation_range=20,      # Rotacionar imagens aleatoriamente até 20 graus
    zoom_range=0.2,         # Aplicar zoom aleatório
    horizontal_flip=True,   # Inverter imagens horizontalmente
)

# Carregar os dados de treino
train_data = datagen.flow_from_directory(
    '/content/dataset',        # Caminho para o dataset
    target_size = (128, 128),  # Redimensionar as imagens
    batch_size = 128,          # Número de amostras por batch
    class_mode ='categorical', # Classificação multiclasses
    subset = 'training',       # Conjunto de treinamento
)

# Carregar os dados de validação
val_data = datagen.flow_from_directory(
    '/content/dataset',
    target_size = (128, 128),
    batch_size = 128,
    class_mode = 'categorical',
    subset = 'validation'
)

# Construindo o modelo
model = Sequential()

K = 3
P = 2

# Primeira camada de convolução + pooling
model.add(Conv2D(32, (K, K), activation='relu', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Segunda camada de convolução + pooling
model.add(Conv2D(64, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Terceira camada de convolução + pooling
model.add(Conv2D(128, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Quarta camada de convolução (opcional)
model.add(Conv2D(256, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Quinta camada de convolução (opcional)
model.add(Conv2D(512, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

model.add(Flatten())

# Exibir o resumo do modelo antes de adicionar a camada densa
model.summary()

# Quatidade de neurônios na camada densa (de acordo com a saída na camada de achatamento (Flatten))
camada_densa = model.layers[-1].output.shape[1]   # Acessa a forma da saída através do atributo 'output'
print(f"Número de neurônios da camada densa: {camada_densa}")

# Deep Learning - Várias camadas densas dentro da camada oculta da Rede Neural (fully connected)
model.add(Dense(camada_densa, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))


# Camada de saída com 19 classes
model.add(Dense(20, activation='softmax'))  # 19 classes (uma para cada pessoa)

# Exibir o resumo do modelo final
model.summary()


# Definindo a taxa de aprendizado da CNN
taxa_de_aprendizado  = 0.0001

# Compilando o modelo
model.compile(Adam(learning_rate = taxa_de_aprendizado),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Treinando o modelo
history = model.fit(
    train_data,
    epochs = 30, # Quantidade de épocas
    validation_data=val_data,
#    callbacks=[early_stop]
)

# Salvando a estrutura do modelo
model_structure = model.to_json()
with open('modelo_classificacao_pessoas.json', 'w') as json_file:
    json_file.write(model_structure)

# Salvando os pesos do modelo
model.save_weights('modelo_classificacao_pessoas.weights.h5')

# Salvando os rótulos em um arquivo TXT
labels = train_data.class_indices  # Obter os rótulos do dataset
with open('rotulos.txt', 'w') as txt_file:
    for label, index in labels.items():
        txt_file.write(f"{label}: {index}\n")  # Salva o rótulo e seu índice


Found 3200 images belonging to 20 classes.
Found 800 images belonging to 20 classes.


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_41 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_39               │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_39 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_42 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_40               │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_40 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_43 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_41               │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_41 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_44 (Conv2D)                   │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_42               │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_42 (MaxPooling2D)      │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_45 (Conv2D)                   │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_43               │ (None, 4, 4, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_43 (MaxPooling2D)      │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 2048)                │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,572,544 (6.00 MB)

 Trainable params: 1,570,560 (5.99 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_41 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_39               │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_39 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_42 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_40               │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_40 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_43 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_41               │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_41 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_44 (Conv2D)                   │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_42               │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_42 (MaxPooling2D)      │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_45 (Conv2D)                   │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_43               │ (None, 4, 4, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_43 (MaxPooling2D)      │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 2048)                │       4,196,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 512)                 │       1,049,0

 Total params: 6,984,788 (26.64 MB)

 Trainable params: 6,982,804 (26.64 MB)

 Non-trainable params: 1,984 (7.75 KB)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 696ms/step - accuracy: 0.1335 - loss: 3.0066 - val_accuracy: 0.0500 - val_loss: 3.0042
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 665ms/step - accuracy: 0.4495 - loss: 1.7190 - val_accuracy: 0.0500 - val_loss: 3.0582
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 664ms/step - accuracy: 0.6744 - loss: 1.0312 - val_accuracy: 0.0500 - val_loss: 3.2386
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 613ms/step - accuracy: 0.8048 - loss: 0.6160 - val_accuracy: 0.0500 - val_loss: 3.5211
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 596ms/step - accuracy: 0.8548 - loss: 0.4248 - val_accuracy: 0.0500 - val_loss: 3.9186
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 580ms/step - accuracy: 0.9316 - loss: 0.2347 - val_accuracy: 0.0500 - val_loss: 4.6270
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 576ms/step - accuracy: 0.9438 - loss: 0.1827 - val_accuracy: 0.0500 - val_loss: 5.0076
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 585ms/step - accuracy: 0.9615 - loss: 0.1397 - val_accu